In [1]:
import functools
import pprint as pp
import shlex
import os
import inspect
import ast
from subprocess import run

In [2]:
def timing(f):
    import time
    @functools.wraps(f)
    def inner(*args, **kwargs):
        start = time.perf_counter()
        value = f(*args, **kwargs)
        end = time.perf_counter()
        print(f"Finished {f.__name__} in {end - start:.4f} seconds.")
        return value
    return inner

In [27]:
def profiler_kind(kind):
    @functools.wraps(kind)
    def usage_profiler(import_classes=""):
        def wrapper(f):
            def inner(*args, **kwargs):

                def unwrap_source_code(g):
                    sourcelines = inspect.getsourcelines(g)
                    for idx, i in enumerate(sourcelines[0]):
                        if i.startswith('def'):            
                            return "".join(sourcelines[0][idx:])

                # Writing f to a temporary file.

                temp = './temp_profiling.py'
                with open(temp, 'w+') as g:
                    decorator = '@profile'
                    value = f"""{import_classes}\nimport ast\nimport argparse\nimport sys\nimport json\n\n\n{decorator}\n{unwrap_source_code(f)}"""
                    g.write(value)
                    g.write('\n')

                    main_exec = f"""if __name__ == "__main__":     
            parser = argparse.ArgumentParser()
            parser.add_argument('-dummy_pos', nargs='*')
            parser.add_argument('--dummy_optional', nargs="?")    
            args = parser.parse_args()

            star_args = args.dummy_pos
            optional_args = args.dummy_optional

            if optional_args:
                optional_args = optional_args.replace("\'", '"')
                optional_args = json.loads(json.loads(json.dumps(optional_args)))
            else:
                optional_args = {{}} 

            dummy_args = []
            for arg in star_args:
                if arg.isnumeric():
                    dummy_args.append(ast.literal_eval(arg))
                else:
                    dummy_args.append(arg)


            {f.__name__}(*dummy_args, **optional_args)
                            """
                    g.write(main_exec)

                cmd = f'{kind()} {temp}'
                if args:
                    args_split = [f'"{str(i)}"' for i in args]
                    second_split =  "-dummy_pos" + " " + " ".join(args_split)

                    cmd = cmd + " " + second_split

                if kwargs:
                    cmd = cmd + " " + '--dummy_optional' + " " + " ".join([f'"{ {k : str(v) for k, v in kwargs.items()} }"'])

                print(cmd)
                stats = run(cmd, capture_output=True).stdout.decode('utf-8')


                # TODO: We might want to have also 
                # a `debug` argument, where we don't delete
                # the temporary file.

               
                print(stats)
                os.remove(temp)
               
                if usage_profiler.__name__ == 'cpu_profiler':
                    os.remove(temp + '.lprof')

                return f(*args, **kwargs)
            return inner
        return wrapper
    return usage_profiler


@profiler_kind
def ram_profiler():
    return 'python -m memory_profiler'


@profiler_kind
def cpu_profiler():
    return 'kernprof -lv'

In [30]:
@ram_profiler()
def my_func(x, y):
    a = [1] * (10 ** x)
    b = [2] * (1 * 10 ** y)
    c = a + b
    del c
    return a



my_func(2, 3);

python -m memory_profiler ./temp_profiling.py -dummy_pos "2" "3"
Filename: ./temp_profiling.py

Line #    Mem usage    Increment   Line Contents
     8   40.254 MiB   40.254 MiB   @profile
     9                             def my_func(x, y):
    10   40.254 MiB    0.000 MiB       a = [1] * (10 ** x)
    11   40.254 MiB    0.000 MiB       b = [2] * (1 * 10 ** y)
    12   40.254 MiB    0.000 MiB       c = a + b
    13   40.254 MiB    0.000 MiB       del c
    14   40.254 MiB    0.000 MiB       return a





In [31]:
@cpu_profiler()
def custom_string_function(x, y):
    print (x * 2)
    print( y * 3)
    return None

custom_string_function('Thanos', 3)

kernprof -lv ./temp_profiling.py -dummy_pos "Thanos" "3"
ThanosThanos
9
Wrote profile results to temp_profiling.py.lprof
Timer unit: 1e-07 s

Total time: 9.3e-06 s
File: ./temp_profiling.py
Function: custom_string_function at line 8

Line #      Hits         Time  Per Hit   % Time  Line Contents
     8                                           @profile
     9                                           def custom_string_function(x, y):
    10         1         70.0     70.0     75.3      print (x * 2)
    11         1         20.0     20.0     21.5      print( y * 3)
    12         1          3.0      3.0      3.2      return None


ThanosThanos
9


In [6]:
from custom_classes import Thanos

@cpu_profiler("from custom_classes import Thanos")
def test_class_function(a, b):
    obj = Thanos(a)
    res = obj.custom_function(b - 1)
    c = [2] * 10**b
    return res

test_class_function(1, 2);

kernprof -lv ./temp_profiling.py -dummy_pos "1" "2"
Wrote profile results to temp_profiling.py.lprof
Timer unit: 1e-07 s

Total time: 7.8e-06 s
File: ./temp_profiling.py
Function: test_class_function at line 8

Line #      Hits         Time  Per Hit   % Time  Line Contents
     8                                           @profile
     9                                           def test_class_function(a, b):
    10         1         42.0     42.0     53.8      obj = Thanos(a)
    11         1         24.0     24.0     30.8      res = obj.custom_function(b - 1)
    12         1         10.0     10.0     12.8      c = [2] * 10**b
    13         1          2.0      2.0      2.6      return res




#  CPU profiler

The following lines could be used only for CPU profiling without writing to an external file, but a similar appraoch **cannot** be used for memory profiling

In [32]:
from decorator import decorator
from line_profiler import LineProfiler

@decorator
def profile_each_line(func, *args, **kwargs):
    profiler = LineProfiler()
    profiled_func = profiler(func)
    try:
        profiled_func(*args, **kwargs)
    finally:
        profiler.print_stats()

In [33]:
@profile_each_line
def test_class_function(a, b):
    obj = Thanos(a)
    res = obj.custom_function(b - 1)
    c = [2] * 10**b
    return res

In [34]:
test_class_function(1, 2)

Timer unit: 1e-07 s

Total time: 1.02e-05 s
File: <ipython-input-33-c3d7b76f3ec2>
Function: test_class_function at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           @profile_each_line
     2                                           def test_class_function(a, b):
     3         1         46.0     46.0     45.1      obj = Thanos(a)
     4         1         36.0     36.0     35.3      res = obj.custom_function(b - 1)
     5         1         16.0     16.0     15.7      c = [2] * 10**b
     6         1          4.0      4.0      3.9      return res

